In [3]:
import pandas as pd 
import requests
from pprint import pprint
import time 
# from api_key import api_key
import sqlite3
import datetime

In [4]:
#Load in CSV
df = pd.read_csv("nyc_traffic_data_2019.csv")

In [5]:
#Look at info 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5841 entries, 0 to 5840
Data columns (total 37 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     5841 non-null   int64  
 1   ID             5841 non-null   int64  
 2   Segment_ID     5841 non-null   int64  
 3   Roadway_Name   5841 non-null   object 
 4   From           5841 non-null   object 
 5   To             5841 non-null   object 
 6   Direction      5841 non-null   object 
 7   Date           5841 non-null   object 
 8   12:00-1:00 AM  5841 non-null   float64
 9   1:00-2:00AM    5841 non-null   float64
 10  2:00-3:00AM    5841 non-null   float64
 11  3:00-4:00AM    5841 non-null   float64
 12  4:00-5:00AM    5841 non-null   float64
 13  5:00-6:00AM    5841 non-null   float64
 14  6:00-7:00AM    5841 non-null   float64
 15  7:00-8:00AM    5841 non-null   float64
 16  8:00-9:00AM    5841 non-null   float64
 17  9:00-10:00AM   5841 non-null   float64
 18  10:00-11

In [6]:
#Recast date object as a datetime object 
df["Date"] = pd.to_datetime(df["Date"])

In [7]:
df["Date"].head()

0   2019-10-06
1   2019-10-07
2   2019-10-08
3   2019-10-09
4   2019-10-10
Name: Date, dtype: datetime64[ns]

In [8]:
#Look only at November 
mask = (df['Date'] > '2019-10-01') & (df['Date'] <= '2019-10-31')
october_df = df.loc[mask]

In [9]:
#Check dataframe
october_df.head(10)

,Unnamed: 0,ID,Segment_ID,Roadway_Name,From,To,Direction,Date,12:00-1:00 AM,1:00-2:00AM,...,7:00-8:00PM,8:00-9:00PM,9:00-10:00PM,10:00-11:00PM,11:00-12:00AM,latitude,longitude,Month,Day,Year
0,793,38,130569,32+Ave,86+St,Alley,WB,2019-10-06,127.0,114.0,...,187,133,123,126,73,40.7580867,-73.882022,10,6,2019
1,1494,38,130569,32+Ave,86+St,Alley,WB,2019-10-07,68.0,40.0,...,197,167,117,102,79,40.7580867,-73.882022,10,7,2019
2,2006,38,130569,32+Ave,86+St,Alley,WB,2019-10-08,83.0,42.0,...,188,171,127,100,93,40.7580867,-73.882022,10,8,2019
3,2616,38,130569,32+Ave,86+St,Alley,WB,2019-10-09,57.0,39.0,...,346,160,97,117,96,40.7580867,-73.882022,10,9,2019
4,2940,38,130569,32+Ave,86+St,Alley,WB,2019-10-10,72.0,45.0,...,288,176,161,101,103,40.7580867,-73.882022,10,10,2019
5,3071,38,130569,32+Ave,86+St,Alley,WB,2019-10-11,74.0,76.0,...,292,273,211,166,195,40.7580867,-73.882022,10,11,2019
6,3362,38,130569,32+Ave,86+St,Alley,WB,2019-10-12,133.0,83.0,...,197,152,162,171,144,40.7580867,-73.882022,10,12,2019
7,3849,38,130569,32+Ave,86+St,Alley,WB,2019-10-13,146.0,125.0,...,216,175,138,124,84,40.7580867,-73.882022,10,13,2019
8,4203,85,56878,Archer+Ave,143+St,144+Pl,EB,2019-10-10,108.0,50.0,...,182,188,229,203,148,40.6995881,-73.810606,10,10,2019
9,4558,85,56878,Archer+Ave,143+St,144+Pl,EB,2019-10-11,116.0,74.0,...,207,181,218,197,163,40.6995881,-73.810606,10,11,2019


In [10]:
#Create daily total count: 
october_df["daily_total"] = df.iloc[:, 7:31].sum(axis=1)

In [13]:
totals = october_df.groupby("Segment_ID").sum()

In [17]:
totals.reset_index(inplace=True)

In [22]:
#Find top 10 intersections in october 
tp10 = totals.nlargest(10, ["daily_total"])

In [26]:
tp10_dict = dict(zip(tp10.Segment_ID, tp10.daily_total))

In [27]:
tp10_dict

{158938: 550750.0,
 64059: 346751.0,
 32329: 337027.0,
 9009697: 328747.0,
 143526: 313441.0,
 249813: 310638.0,
 28973: 310330.0,
 22096: 303341.0,
 229411: 302821.0,
 36112: 302136.0}

In [30]:
intersection_list = []
for key in tp10_dict:
    a = df.loc[df["Segment_ID"] == key]
    intersection_list.append(a["latitude"])
    intersection_list.append(a["longitude"])

In [31]:
intersection_list

[5652    40.7755364
 5653    40.7755364
 5654    40.7755364
 5655    40.7755364
 5656    40.7755364
 5657    40.7755364
 5658    40.7755364
 5659    40.7755364
 5660    40.7755364
 5661    40.7755364
 5662    40.7755364
 5663    40.7755364
 5664    40.7755364
 5665    40.7755364
 5666    40.7755364
 5667    40.7755364
 5668    40.7755364
 5669    40.7755364
 Name: latitude, dtype: object,
 5652   -73.991474
 5653   -73.991474
 5654   -73.991474
 5655   -73.991474
 5656   -73.991474
 5657   -73.991474
 5658   -73.991474
 5659   -73.991474
 5660   -73.991474
 5661   -73.991474
 5662   -73.991474
 5663   -73.991474
 5664   -73.991474
 5665   -73.991474
 5666   -73.991474
 5667   -73.991474
 5668   -73.991474
 5669   -73.991474
 Name: longitude, dtype: float64,
 4995    40.66555820000001
 4996    40.66555820000001
 4997    40.66555820000001
 4998    40.66555820000001
 4999    40.66555820000001
 5000    40.66555820000001
 5001    40.66555820000001
 5002    40.66555820000001
 5003    40.6655